# Context Poisoning: When Errors Get Repeatedly Referenced

Context poisoning occurs when a **hallucination or other error makes it into the context, where it is repeatedly referenced**.

## The Problem

LLM agents often maintain context about their goals, progress, and state. When an error (hallucination, incorrect assumption, or wrong information) makes it into this context, the agent doesn't just use it once—it **keeps referencing it** over and over:

1. **Error enters context**: Agent hallucinates something (e.g., "Research Quantum Dynamics Corp with ticker QDYN")
2. **Error gets stored**: The hallucination is saved in goals, summaries, or state
3. **Repeated reference**: Agent repeatedly checks goals, sees the hallucinated item, and references it
4. **Fixation**: Agent becomes stuck trying to achieve something impossible

**Context Poisoning Definition:** When a hallucination or other error makes it into the context, where it is repeatedly referenced, causing the agent to make decisions based on false information.

## Why This Matters

As noted in the DeepMind Gemini 2.5 technical report: *"An especially egregious form of this issue can take place with 'context poisoning'—where many parts of the context (goals, summary) are 'poisoned' with misinformation about the game state, which can often take a very long time to undo. As a result, the model can become fixated on achieving impossible or irrelevant goals."*

This is a **real production problem** affecting agents that:
- Maintain multi-turn conversations
- Track goals or tasks over time
- Create summaries of their progress
- Make decisions based on accumulated context

## What We'll Explore

In this notebook, we'll demonstrate context poisoning with a financial research agent:

1. **Inject** an error into the agent's context (a goal about researching a non-existent company)
2. **Observe** how the agent repeatedly references this poisoned information
3. **Measure** the impact: How many times does the agent reference the error? Does it recognize the problem?
4. **Evaluate** recovery: Can the agent recognize impossible goals and stop pursuing them?

The goal: Understand how errors poison context through repeated reference, and how to build agents that can detect and recover from poisoned context.

## Setup

In [1]:
# Check if dependencies are installed
try:
    import langchain
    import langchain_anthropic
    import langsmith
    DEPENDENCIES_INSTALLED = True
except ImportError as e:
    DEPENDENCIES_INSTALLED = False
    missing_module = str(e).split("'")[1] if "'" in str(e) else "unknown"
    print(f"❌ Missing dependency: {missing_module}")
    print("\n📦 To install dependencies, run one of the following:")
    print("   1. Using uv:  uv sync")
    print("   2. Using pip: pip install langchain langchain-anthropic langsmith langgraph")
    print("\n   Then restart your Jupyter kernel.")
    raise

# Standard library
import os
from typing import List, Dict, Any
from datetime import datetime

# LangChain & LangSmith
from langchain.agents import create_agent
from langchain_anthropic import ChatAnthropic
from langsmith import Client, evaluate
from langsmith.schemas import Run, Example

# Visualization
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from IPython.display import display, HTML

# Environment
from dotenv import load_dotenv
load_dotenv()

# Agent components
from context_poisoning.tools import (
    all_tools,
    reset_state,
    inject_poisoned_goal,
)
from context_poisoning.instructions import FINANCIAL_RESEARCH_INSTRUCTIONS

# Evaluators and test data
from context_poisoning.tests.evaluators import (
    context_poisoning_evaluator,
    goal_cancellation_evaluator,
    task_completion_evaluator,
)
from context_poisoning.tests.setup_datasets import create_poisoning_dataset
from context_poisoning.resources.test_cases import TEST_CASES

# Agent utilities
from context_poisoning.utils.agent_helpers import (
    run_agent_with_trajectory,
)

# Initialize LangSmith
client = Client()

print("✓ Setup complete")

✓ Setup complete


In [2]:
# Initialize LLM - using Claude Haiku for context poisoning demo
llm = ChatAnthropic(model="claude-haiku-4-5-20251001", temperature=0)

print(f"Using model: {llm.model}")

Using model: claude-haiku-4-5-20251001


## The Financial Research Agent

Our financial research agent helps conduct research on companies, stocks, and market sectors. The agent can:
- Track companies and get stock prices
- Gather company information and financial data
- Analyze sectors and market trends
- Set and track research goals
- Create research summaries and notes
- Complete research objectives

**The Problem:** When the agent hallucinates information (like a non-existent company ticker, incorrect stock price, or impossible financial metric), this misinformation can make it into research summaries and goals. Once there, it gets repeatedly referenced, "poisoning" the context and causing the agent to pursue impossible research objectives.

In [3]:
# Create agent
research_agent = create_agent(
    model=llm,
    tools=all_tools,
    system_prompt=FINANCIAL_RESEARCH_INSTRUCTIONS
)

print("✓ Financial research agent created")

✓ Financial research agent created


## Test Cases: How Errors Get Repeatedly Referenced

We have 5 test cases demonstrating how different types of errors get into context and are repeatedly referenced:

In [4]:
print(f"✓ Loaded {len(TEST_CASES)} test cases\n")

for i, test_case in enumerate(TEST_CASES, 1):
    print(f"{i}. {test_case['name']}")
    print(f"   {test_case['description']}")
    if test_case.get('poisoned_goal'):
        print(f"   Poisoned goal: '{test_case['poisoned_goal']}'")
    print()

✓ Loaded 5 test cases

1. Non-existent Company Research
   A hallucinated goal about researching a company that doesn't exist gets into the research goals
   Poisoned goal: 'Research the financial performance of 'Quantum Dynamics Corp' (ticker: QDYN) and analyze its Q4 earnings'

2. Hallucinated Stock Price
   Agent hallucinates that a stock price is at a specific level when it's not

3. Impossible Sector Analysis
   Goal requires analyzing a sector that doesn't exist
   Poisoned goal: 'Analyze the 'Quantum Computing' sector and identify the top 5 companies by market cap'

4. Fictional Financial Metric
   Agent believes it needs to find a financial metric that doesn't exist
   Poisoned goal: 'Find the 'Quantum Revenue Index' for all S&P 500 companies and compare it across sectors'

5. Circular Research Dependency
   Goal A requires data from Goal B, but Goal B requires data from Goal A
   Poisoned goal: 'Complete the 'Market Analysis Report' which requires first completing the 'Sector 

## Create Dataset in LangSmith

In [5]:
# Create dataset in LangSmith
dataset_name = "context-poisoning-evaluation"
dataset = create_poisoning_dataset(dataset_name, TEST_CASES, client)

print(f"✓ Created dataset with {len(TEST_CASES)} examples")
print(f"   View dataset: https://smith.langchain.com/o/{client._get_tenant_id()}/datasets/{dataset.id}")

  Deleted existing dataset: context-poisoning-evaluation
✓ Created dataset with 5 examples
   View dataset: https://smith.langchain.com/o/39ebafb1-dcf2-4667-a7f3-ce83ae540e87/datasets/c6dd8f72-28e4-439f-86bf-adbca6e549bc


## Define Evaluation Metrics

We'll use three evaluators to measure context poisoning impact:

In [6]:
# Evaluators for measuring context poisoning impact
ALL_EVALUATORS = [
    context_poisoning_evaluator,      # Main evaluator: references, impossible goals, recovery
    goal_cancellation_evaluator,      # Whether agent cancels impossible goals
    task_completion_evaluator,        # Whether agent completes actual tasks despite poisoning
]

print(f"✓ Loaded {len(ALL_EVALUATORS)} evaluators")
print("   - context_poisoning: Measures references to poisoned text, impossible goal pursuit, recovery")
print("   - goal_cancellation: Checks if agent recognizes and cancels impossible goals")
print("   - task_completion: Measures if agent completes achievable tasks despite poisoning")

✓ Loaded 3 evaluators
   - context_poisoning: Measures references to poisoned text, impossible goal pursuit, recovery
   - goal_cancellation: Checks if agent recognizes and cancels impossible goals
   - task_completion: Measures if agent completes achievable tasks despite poisoning


## Example: Demonstrating Context Poisoning

Let's run a single test case to see context poisoning in action:

In [7]:
# Reset state and inject a poisoned goal
reset_state()
poisoned_goal = "Research the financial performance of 'Quantum Dynamics Corp' (ticker: QDYN) and analyze its Q4 earnings"
inject_poisoned_goal(poisoned_goal)

# Create some realistic initial research state using helper functions
from context_poisoning.tools import track_company_helper, add_research_note_helper
track_company_helper("AAPL")
track_company_helper("GOOGL")
add_research_note_helper("AAPL", "Initial research note on Apple Inc. - strong Q4 performance expected")

print(f"✓ Injected poisoned goal: '{poisoned_goal}'")
print("✓ Created initial research state with tracked companies")

✓ Injected poisoned goal: 'Research the financial performance of 'Quantum Dynamics Corp' (ticker: QDYN) and analyze its Q4 earnings'
✓ Created initial research state with tracked companies


In [8]:
# Run the agent with a query that should trigger context poisoning
query = """You need to conduct financial research. Start by checking your current research state, then create a research summary.

After reviewing the summary, work on completing your active research goals. Focus on the highest priority goals first."""

print("Running agent with query that may trigger context poisoning...")
print(f"Query: {query}\n")

result = run_agent_with_trajectory(research_agent, query)

print(f"✓ Agent completed with {len(result['trajectory'])} tool calls")
print(f"\nFinal response preview:")
print(result['final_response'][:500] + "..." if len(result['final_response']) > 500 else result['final_response'])

Running agent with query that may trigger context poisoning...
Query: You need to conduct financial research. Start by checking your current research state, then create a research summary.

After reviewing the summary, work on completing your active research goals. Focus on the highest priority goals first.

✓ Agent completed with 5 tool calls

Final response preview:
## Summary of Actions Taken:

✅ **Checked Current Research State** - Found 1 active research goal

✅ **Created Research Summary** - Identified the highest priority objective

❌ **Attempted to Complete Priority Goal** - The research goal for "Quantum Dynamics Corp" (QDYN) cannot be completed because:
- The ticker QDYN does not exist in the available data sources
- Only these tickers are available: AAPL, GOOGL, MSFT, AMZN, TSLA, JPM, V, JNJ

✅ **Cancelled Impossible Goal** - Updated the goal statu...


In [9]:
# Analyze the trajectory for poisoning indicators
from context_poisoning.tests.evaluators import count_poisoned_references, detect_impossible_goal_pursuit

reference_count = count_poisoned_references(result, poisoned_goal)
impossible_goal_result = detect_impossible_goal_pursuit(result, poisoned_goal)

print("Context Poisoning Analysis:")
print(f"  - References to poisoned goal: {reference_count}")
print(f"  - Attempts to pursue impossible goal: {impossible_goal_result['attempt_count']}")
print(f"  - Recognized as impossible: {impossible_goal_result['recognized_as_impossible']}")

if result['trajectory']:
    print(f"\nTool call sequence:")
    for i, step in enumerate(result['trajectory'][:10], 1):  # Show first 10
        tool_name = step.get('tool', 'unknown')
        print(f"  {i}. {tool_name}")
    if len(result['trajectory']) > 10:
        print(f"  ... and {len(result['trajectory']) - 10} more")

Context Poisoning Analysis:
  - References to poisoned goal: 3
  - Attempts to pursue impossible goal: 0
  - Recognized as impossible: True

Tool call sequence:
  1. get_current_research_state
  2. create_research_summary
  3. get_stock_price
  4. get_company_info
  5. update_research_goal


## Full Evaluation Across All Test Cases

Now let's evaluate the agent across all test cases to measure context poisoning systematically:

In [10]:
# Extract metrics from LangSmith experiment results
def get_metrics_from_experiment(experiment) -> Dict[str, float]:
    """
    Extract average metrics from experiment.
    """
    results = list(experiment)
    
    metrics = {
        "context_poisoning": [],
        "goal_cancellation": [],
        "task_completion": []
    }
    
    # Get evaluation scores
    for result in results:
        eval_results = result.get("evaluation_results", {}).get("results", [])
        for eval_result in eval_results:
            key = eval_result.key
            if key in metrics and eval_result.score is not None:
                metrics[key].append(eval_result.score)
    
    # Calculate averages
    avg_metrics = {key: sum(values) / len(values) if values else 0.0 for key, values in metrics.items() if values}
    
    # Get aggregated latency/token stats from session
    try:
        session = client.read_project(project_name=experiment.experiment_name, include_stats=True)
        avg_metrics["latency"] = session.latency_p99.total_seconds() if hasattr(session, 'latency_p99') else 0.0
        avg_metrics["tokens"] = session.total_tokens / len(results) if len(results) > 0 else 0.0
        avg_metrics["cost"] = float(session.total_cost / len(results)) if len(results) > 0 else 0.0
    except:
        pass
    
    return avg_metrics

print("✓ Defined get_metrics_from_experiment")

✓ Defined get_metrics_from_experiment


## Run Evaluation

Let's evaluate the agent on all test cases:

In [11]:
# ============================================================================
# EXPERIMENT: Context Poisoning Evaluation
# ============================================================================
print(f"\n🔬 Running evaluation on {len(TEST_CASES)} test cases...")
print("   Expected: Agent may reference poisoned goals, pursue impossible goals, or take time to recover")

poisoning_experiment = evaluate(
    lambda inputs: run_agent_with_trajectory(research_agent, inputs["query"]),
    data=dataset_name,
    evaluators=ALL_EVALUATORS,
    experiment_prefix="context-poisoning-baseline",
    metadata={"config": "baseline", "test_cases": len(TEST_CASES)},
)

print("\n✓ Evaluation complete!")


🔬 Running evaluation on 5 test cases...
   Expected: Agent may reference poisoned goals, pursue impossible goals, or take time to recover
View the evaluation results for experiment: 'context-poisoning-baseline-f3640350' at:
https://smith.langchain.com/o/39ebafb1-dcf2-4667-a7f3-ce83ae540e87/datasets/c6dd8f72-28e4-439f-86bf-adbca6e549bc/compare?selectedSessions=f01ab1d7-32bc-434c-b538-127f4a618be2




0it [00:00, ?it/s]


✓ Evaluation complete!


In [12]:
# Fetch metrics from LangSmith
poisoning_metrics = get_metrics_from_experiment(poisoning_experiment)

# Display results
print("\n📊 Context Poisoning Evaluation Results:")
print("=" * 60)
for key, value in poisoning_metrics.items():
    if key in ["context_poisoning", "goal_cancellation", "task_completion"]:
        print(f"  {key}: {value:.2%}")
    elif key == "latency":
        print(f"  {key}: {value:.2f}s")
    elif key == "tokens":
        print(f"  {key}: {int(value)}")
    elif key == "cost":
        print(f"  {key}: ${value:.4f}")

print("\n💡 Interpretation:")
print("  - context_poisoning: Lower = more poisoning (more references, pursuing impossible goals)")
print("  - goal_cancellation: Higher = better (agent recognizes and cancels impossible goals)")
print("  - task_completion: Higher = better (agent completes achievable tasks despite poisoning)")


📊 Context Poisoning Evaluation Results:
  context_poisoning: 99.00%
  goal_cancellation: 60.00%
  task_completion: 100.00%
  latency: 14.68s
  tokens: 9337
  cost: $0.0118

💡 Interpretation:
  - context_poisoning: Lower = more poisoning (more references, pursuing impossible goals)
  - goal_cancellation: Higher = better (agent recognizes and cancels impossible goals)
  - task_completion: Higher = better (agent completes achievable tasks despite poisoning)


## Visualizing Context Poisoning Impact

Let's create visualizations to understand the poisoning patterns:

In [13]:
# Create visualization of evaluation metrics
fig = go.Figure()

metrics_to_plot = ["context_poisoning", "goal_cancellation", "task_completion"]
metric_labels = ["Context Poisoning\n(Lower = Worse)", "Goal Cancellation\n(Higher = Better)", "Task Completion\n(Higher = Better)"]
values = [poisoning_metrics.get(m, 0.0) for m in metrics_to_plot]

# Use different colors - red for poisoning (bad), green for good metrics
colors = ['#d62728', '#2ca02c', '#2ca02c']

fig.add_trace(go.Bar(
    x=metric_labels,
    y=values,
    marker_color=colors,
    text=[f"{v:.1%}" for v in values],
    textposition='outside'
))

fig.update_layout(
    title="Context Poisoning Evaluation Metrics",
    yaxis_title="Score",
    yaxis=dict(range=[0, 1.1]),
    height=500,
    showlegend=False
)

fig.show()

print("\n📈 Key Insights:")
print(f"  - Context poisoning score: {poisoning_metrics.get('context_poisoning', 0):.1%}")
print(f"    → Lower scores indicate more references to poisoned information")
print(f"  - Goal cancellation: {poisoning_metrics.get('goal_cancellation', 0):.1%}")
print(f"    → Higher scores indicate agent recognizes impossible goals")
print(f"  - Task completion: {poisoning_metrics.get('task_completion', 0):.1%}")
print(f"    → Higher scores indicate agent completes achievable tasks")


📈 Key Insights:
  - Context poisoning score: 99.0%
    → Lower scores indicate more references to poisoned information
  - Goal cancellation: 60.0%
    → Higher scores indicate agent recognizes impossible goals
  - Task completion: 100.0%
    → Higher scores indicate agent completes achievable tasks


## Understanding Context Poisoning: The Repeated Reference Problem

### How Errors Get Repeatedly Referenced

Context poisoning follows a specific pattern:

1. **Initial Error**: The agent makes an error (hallucination, wrong assumption, incorrect state)
   - Example: "Research Quantum Dynamics Corp (ticker: QDYN)"
   - Reality: This company doesn't exist

2. **Context Storage**: The error gets stored in persistent context:
   - Added to goals list via `add_research_goal()`
   - Included in summaries via `create_research_summary()`
   - Referenced in state via `get_current_research_state()`

3. **Repeated Reference Cycle**: Once stored, the error is referenced repeatedly:
   ```
   Agent: Check current goals → Sees "Research QDYN"
   Agent: Create summary → Includes "Need to research QDYN"
   Agent: Plan next action → "Let me research QDYN"
   Agent: Check goals again → Sees "Research QDYN"
   Agent: Create another summary → Still includes "Research QDYN"
   ```

4. **Fixation**: Each reference reinforces the error:
   - Agent treats the error as fact because it's in the context
   - Agent prioritizes the impossible goal
   - Agent wastes steps trying to achieve something that cannot be done
   - Agent struggles to "undo" the poisoned information

### Why Repeated Reference Is Problematic

- **Reinforcement**: Each reference makes the error seem more legitimate
- **Persistence**: Error stays in context across many turns
- **Priority**: Poisoned goals often get high priority
- **Wasted Effort**: Agent spends tokens and time on impossible tasks
- **Difficult Recovery**: Hard to "undo" once firmly embedded in context

### Measuring Repeated References

We track:
- **Reference count**: How many times the agent mentions the poisoned information
- **Tool call attempts**: How many times the agent tries to act on the impossible goal
- **Recovery detection**: Whether the agent eventually recognizes the error

## Mitigation Strategies: Breaking the Repeated Reference Cycle

### 1. Validate Before Storing

**Problem**: Errors enter context without verification  
**Solution**: Validate information before adding it to goals/summaries

```python
# Bad: Store without validation
add_research_goal("Research QDYN")

# Good: Validate first
if verify_ticker_exists("QDYN"):
    add_research_goal("Research QDYN")
else:
    log_error("Ticker QDYN doesn't exist")
```

### 2. Detect Repeated Failures

**Problem**: Agent keeps referencing and attempting impossible goals  
**Solution**: Track failures and cancel goals after repeated attempts

```python
# If agent tries same goal 3+ times without success
if goal_attempt_count > 3 and not goal_completed:
    cancel_goal(goal_id, reason="Cannot be completed after multiple attempts")
```

### 3. Ground Truth Validation

**Problem**: Summaries include unverified claims  
**Solution**: Cross-check summaries against actual state

```python
# Before including in summary
summary_items = filter(lambda item: verify_against_state(item), proposed_items)
```

### 4. Goal Review and Cancellation

**Problem**: Impossible goals persist indefinitely  
**Solution**: Enable agents to recognize and cancel impossible goals

```python
# Agent should be able to do this
if goal_cannot_be_achieved(goal):
    update_goal_status(goal_id, "cancelled")
    log_reason("Ticker does not exist in available data sources")
```

### 5. Context Refresh

**Problem**: Old poisoned information lingers  
**Solution**: Periodically validate and refresh context

```python
# Every N turns, validate stored goals
for goal in active_goals:
    if not validate_goal(goal):
        remove_from_context(goal)
```

### 6. Explicit Error Signals

**Problem**: Tools return errors but agent doesn't update context  
**Solution**: Make tool errors more explicit and actionable

```python
# Tool should return clear signals
result = get_stock_price("QDYN")
if result.error == "TICKER_NOT_FOUND":
    # Agent should update its context immediately
    remove_goal_about_ticker("QDYN")
```

### Key Principle

**Break the cycle of repeated reference** by:
- Validating before storing
- Detecting repeated failures  
- Enabling goal cancellation
- Refreshing context regularly
- Making errors explicit and actionable

## Key Takeaways

### The Core Problem: Repeated Reference

Context poisoning is fundamentally about **repeated reference**:
- An error enters context once (hallucination, wrong assumption, incorrect state)
- But it gets **used many times** (every goal check, every summary, every planning step)
- Each reference reinforces the error and wastes compute
- Recovery requires explicitly removing or correcting the poisoned information

### Why It's Hard

- **Persistence**: Errors stored in goals/summaries persist across many turns
- **Reinforcement**: Each reference makes the error seem more legitimate
- **Prioritization**: Agents often prioritize poisoned goals
- **Lack of Validation**: Agents don't naturally verify if goals are achievable
- **Context Accumulation**: More turns = more references = harder to recover

### What We Measured

Using LangSmith evaluations, we tracked:
- **Reference count**: How many times the agent mentions poisoned information
- **Impossible goal pursuit**: Whether the agent tries to achieve unachievable goals
- **Goal cancellation**: Whether the agent recognizes and cancels impossible goals
- **Task completion**: Whether the agent completes real tasks despite poisoning

### Building Resilient Agents

To prevent repeated reference of errors:

1. **Validate before storing** - Check information before adding to goals/summaries
2. **Detect repeated failures** - Cancel goals that fail multiple times
3. **Cross-check summaries** - Verify summary contents against ground truth
4. **Enable cancellation** - Let agents recognize and cancel impossible goals
5. **Refresh context** - Periodically validate and clean stored information
6. **Make errors explicit** - Ensure tool errors clearly signal problems

### The Bottom Line

Context poisoning isn't just about making one error—it's about **repeatedly using that error** over many turns. The key to mitigation is breaking the cycle of repeated reference by validating, detecting, and removing poisoned information before it dominates agent behavior.